#Trabajo Final Grupo Nº 2

Rafael Perez	rafaelperezctes@gmail.com
Olinca Ayala	olincayalan88@gmail.com
###@Created on Dec Sat 2 16:57:20 2023
###@author: Grupo Nº 2


# Modulo A I

Este cuaderno realiza la descarga y análisis de secuencias nucleotídicas de integronas de Shewanella utilizando Biopython.


In [15]:
# @title Instalación de librerias
import subprocess
import ipywidgets as widgets
from IPython.display import display
import pkg_resources

def instalar_librerias():
    librerias = [
        "import_ipynb",
        "nbconvert",
        "ipywidgets",
        "biopython",
        "pandas",
        "numpy",
        "matplotlib",
        "tqdm",
        "sklearn"]

    barra_progreso = widgets.IntProgress(
        value=0,
        min=0,
        max=len(librerias),
        bar_style='info',
        orientation='horizontal'
    )

    etiqueta = widgets.Label(value='Iniciando instalación...')
    display(widgets.VBox([etiqueta, barra_progreso]))

    # Instalación de fasttree y mafft con apt-get (solo ejecutar en Colab)
    subprocess.run(["apt-get", "install", "fasttree", "mafft", "ghostscript"])

    for libreria in librerias:
        try:
            pkg_resources.require(libreria)
            etiqueta.value = f"{libreria} ya está instalado."
        except pkg_resources.DistributionNotFound:
            etiqueta.value = f"Instalando: {libreria}..."
            subprocess.run(["pip", "install", libreria])
        barra_progreso.value += 1

    etiqueta.value = 'Instalación completada.'
    # importo todas las librerias
# Crear botón
boton_instalar = widgets.Button(description='Instalar Librerías')

# Crear área de salida
output = widgets.Output()
display(output)

# Enlazar el botón con la función de instalación
boton_instalar.on_click(lambda b: instalar_librerias())

# Mostrar botón
#display(boton_instalar)

#instalar_librerias()


Output()

In [16]:
#modelo de árbol de decisión,
#considerando todas las características que se pueden extraer de tus datos, es un proyecto extenso

# paso 1: extracion de caracteristicas

# paso 2: preparar el modelo


In [17]:
#@title modelo de arboles de desicion

def generador_de_etiquetas(conservacion, entropia):
    import numpy as np
    # Definir umbrales
    umbral_conservacion = np.percentile(conservacion, 50)
    umbral_entropia = np.percentile(entropia, 50)

    # Generar etiquetas basadas en los umbrales
    etiquetas = []
    for c, e in zip(conservacion, entropia):
        if c >= umbral_conservacion and e < umbral_entropia:
            etiquetas.append('Clase 1')
        elif c < umbral_conservacion and e >= umbral_entropia:
            etiquetas.append('Clase 2')
        else:
            etiquetas.append('Clase 3')

    return etiquetas

def extraer_caracteristicas_y_guardar(archivo_alineamiento, archivo_salida):
    import pandas as pd
    from Bio import AlignIO
    import numpy as np
    from scipy import stats

    alineamiento = AlignIO.read(archivo_alineamiento, "fasta")
    num_secuencias = len(alineamiento)
    longitud_secuencia = alineamiento.get_alignment_length()

    conservacion = np.zeros(longitud_secuencia)
    entropia = np.zeros(longitud_secuencia)
    for i in range(longitud_secuencia):
        columna = alineamiento[:, i]
        conservacion[i] = columna.count(max(set(columna), key=columna.count)) / num_secuencias
        entropia[i] = stats.entropy(np.unique(columna, return_counts=True)[1], base=2)

    etiquetas = generador_de_etiquetas(conservacion, entropia)

    caracteristicas = pd.DataFrame({
        'Conservacion': conservacion,
        'Entropia': entropia,
        'Etiqueta': etiquetas
    })

    caracteristicas.to_csv(archivo_salida, index=False)

    return caracteristicas

def cargar_caracteristicas_etiquetas(archivo_csv):
    import pandas as pd
    caracteristicas_con_etiquetas = pd.read_csv(archivo_csv)
    X = caracteristicas_con_etiquetas.drop('Etiqueta', axis=1)
    y = caracteristicas_con_etiquetas['Etiqueta']
    return X, y

def dividir_datos_entrenamiento_prueba(X, y, test_size=0.2, random_state=42):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

def entrenar_evaluar_arbol_decision(X_train, y_train, X_test, y_test):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import accuracy_score
    modelo = DecisionTreeClassifier()
    modelo.fit(X_train, y_train)
    predicciones = modelo.predict(X_test)
    precision = accuracy_score(y_test, predicciones)
    print(f"Precisión del modelo: {precision}")
    return modelo

# Ejemplo de cómo usar estas funciones en otro cuaderno
# caracteristicas = extraer_caracteristicas_y_guardar('ruta/del/alineamiento.fasta', 'caracteristicas.csv')
#X, y = cargar_caracteristicas_etiquetas('caracteristicas_con_etiquetas.csv')
#X_train, X_test, y_train, y_test = dividir_datos_entrenamiento_prueba(X, y)
# modelo = entrenar_evaluar_arbol_decision(X_train, y_train, X_test, y_test)

In [18]:
def graficar_arbol_decision(modelo):
    import matplotlib.pyplot as plt
    from sklearn.tree import plot_tree
    plt.figure(figsize=(20,10))
    plot_tree(modelo, filled=True, rounded=True, fontsize=14)
    plt.show()

In [19]:
def main(archivo):
  instalar_librerias()
  X, y = cargar_caracteristicas_etiquetas(archivo)
  X_train, X_test, y_train, y_test = dividir_datos_entrenamiento_prueba(X, y)
  modelo = entrenar_evaluar_arbol_decision(X_train, y_train, X_test, y_test)
  graficar_arbol_decision(modelo)
#main("")